## 1. Full Automatic Using <code>model=nn.Linear()</code>

In [1]:
import torch
import torch.nn as nn



X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
T = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

#--------------------------------------------------------------------------
#Defining the model by taking number of features and number of samples
#No need to intialize the weights manually
n_samples, n_features = X.shape
input_size = n_features
output_size = n_features
model = nn.Linear(input_size, output_size)


#--------------------------------------------------------------------------
#Inferencing variable should also be a tensor to be inputed to model=nn.Linear()
x_test = torch.tensor([5], dtype=torch.float32)
print(f'Prediction before training: f(5) = {model(x_test).item():.3f}')


#--------------------------------------------------------------------------
learning_rate = 0.01 #Increase this and you can see gradient explosion.
n_iters = 40


#--------------------------------------------------------------------------
#Error/Loss and the optimizer where weight updates happen is defined using torch.
#optimizer automatically updates the weights after one training loop iteration.
#No need to do it manually and also to stop autograd from tracking weight update.
Error = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #Instead of simply giving w.


#--------------------------------------------------------------------------
for epoch in range(n_iters):
    # predict = forward pass
    Y = model(X)

    # loss is done using torch.MOSE
    E = Error(T, Y)
    
    # Backward pass is done automatically now.
    E.backward()

    # Weight update is done automatically using optimizer. No need to wrap things in 
    # torch.no_grad()
    optimizer.step()

    # The w.grad where the gradients are accumilated should be turned zero before next iteration.
    optimizer.zero_grad()

    if epoch % 4 == 0:

        #Weights are inserted into model.parameters(). Need to unroll them.
        [w, b] = model.parameters()

        # print("w: ", w)
        # print("w[0]: ", w[0])
        # print("w[0][0]]: ", w[0][0])
        # w is a tensor, that is a list of a list. Hence w[0][0] is needed.

        print(f'epoch {epoch+1}: w = {w[0][0].item():.3f}, loss = {E:.8f}')
     
#--------------------------------------------------------------------------
print(f'Prediction before training: f(5) = {model(x_test).item():.3f}')

Prediction before training: f(5) = -4.611
epoch 1: w = -0.468, loss = 64.46286011
epoch 5: w = 0.670, loss = 15.02735329
epoch 9: w = 1.220, loss = 3.56565261
epoch 13: w = 1.487, loss = 0.90675443
epoch 17: w = 1.616, loss = 0.28849059
epoch 21: w = 1.681, loss = 0.14331460
epoch 25: w = 1.713, loss = 0.10784941
epoch 29: w = 1.730, loss = 0.09785925
epoch 33: w = 1.740, loss = 0.09381659
epoch 37: w = 1.747, loss = 0.09119352
Prediction before training: f(5) = 9.479


## 2. Using Loss and Optimizer from Torch.

In [2]:
import torch
import torch.nn as nn


X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
T = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

#--------------------------------------------------------------------------
# Intializing the weights. This is the variable we are calculating the gradient on.
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True) 


#--------------------------------------------------------------------------
#forward function where the model get implicitly defined.
def forward(x):
    return w*x

#--------------------------------------------------------------------------
#Inferencing variable.
print(f'Prediction before training: f(5) = {forward(5):.3f}')



#--------------------------------------------------------------------------
learning_rate = 0.01 #Increase this and you can see gradient explosion.
n_iters = 50



#--------------------------------------------------------------------------
#Error/Loss and the optimizer where weight updates happen is defined using torch.
#optimizer automatically updates the weights after one training loop iteration.
#No need to do it manually and also to stop autograd from tracking weight update.
Error = nn.MSELoss()
optimizer = torch.optim.SGD([w], lr=learning_rate)



#--------------------------------------------------------------------------
for epoch in range(n_iters):
    # predict = forward pass
    Y = forward(X)

    # loss is done using torch.nn.MSELoss()
    E = Error(T, Y)
    
    # Backward pass is done automatically now.
    E.backward()

    # Weight update is done automatically using optimizer. No need to wrap things in 
    # torch.no_grad()
    optimizer.step()

    # The w.grad where the gradients are accumilated should be turned zero before next iteration.
    optimizer.zero_grad()

    if epoch % 4 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {E:.8f}')
     


#--------------------------------------------------------------------------
print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 5: w = 1.113, loss = 8.17471695
epoch 9: w = 1.537, loss = 2.22753215
epoch 13: w = 1.758, loss = 0.60698116
epoch 17: w = 1.874, loss = 0.16539653
epoch 21: w = 1.934, loss = 0.04506890
epoch 25: w = 1.966, loss = 0.01228084
epoch 29: w = 1.982, loss = 0.00334642
epoch 33: w = 1.991, loss = 0.00091188
epoch 37: w = 1.995, loss = 0.00024848
epoch 41: w = 1.997, loss = 0.00006770
epoch 45: w = 1.999, loss = 0.00001845
epoch 49: w = 1.999, loss = 0.00000503
Prediction after training: f(5) = 9.997


## 3. Using AutoGrad

In [3]:
import torch

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
T = torch.tensor([2, 4, 6, 8], dtype=torch.float32)


#--------------------------------------------------------------------------
# Intializing the weights. This is the variable we are calculating the gradient on.
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True) 


#--------------------------------------------------------------------------
#forward function where the model get implicitly defined.
def forward(x):
    return w*x


#--------------------------------------------------------------------------
#Inferencing variable.
print(f'Prediction before training: f(5) = {forward(5):.3f}')


#--------------------------------------------------------------------------
learning_rate = 0.01 #Increase this and you can see gradient explosion.
n_iters = 50


#--------------------------------------------------------------------------
#Loss function is defined by hand. 
def loss(y, t): #y=prediction, t=target.
    return ((y-t)**2).mean()



#--------------------------------------------------------------------------
for epoch in range(n_iters):
    # predict = forward pass
    Y = forward(X)

    # loss
    E = loss(T, Y)
    
    # Backward pass is done automatically now.
    E.backward()

    # updating weights should not be tracke from the autograd module. This should be done manually.
    with torch.no_grad():
        w -= learning_rate * w.grad

    # The w.grad where the gradients are accumilated should be turned zero before next iteration.
    w.grad.zero_()

    if epoch % 4 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {E:.8f}')
     

#--------------------------------------------------------------------------
print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 5: w = 1.113, loss = 8.17471695
epoch 9: w = 1.537, loss = 2.22753215
epoch 13: w = 1.758, loss = 0.60698116
epoch 17: w = 1.874, loss = 0.16539653
epoch 21: w = 1.934, loss = 0.04506890
epoch 25: w = 1.966, loss = 0.01228084
epoch 29: w = 1.982, loss = 0.00334642
epoch 33: w = 1.991, loss = 0.00091188
epoch 37: w = 1.995, loss = 0.00024848
epoch 41: w = 1.997, loss = 0.00006770
epoch 45: w = 1.999, loss = 0.00001845
epoch 49: w = 1.999, loss = 0.00000503
Prediction after training: f(5) = 9.997


## 4. Done Everything Manually

In [4]:
import numpy as np

X = np.array([1, 2, 3, 4], dtype=np.float32)
T = np.array([2, 4, 6, 8], dtype=np.float32)

#--------------------------------------------------------------------------
w = 0.0


#--------------------------------------------------------------------------
def forward(x):
    return w*x


#--------------------------------------------------------------------------
print(f'Prediction before training: f(5) = {forward(5):.3f}')



#--------------------------------------------------------------------------
learning_rate = 0.01 #Increase this and you can see gradient explosion.
n_iters = 50



#--------------------------------------------------------------------------
def loss(y, t): #y=prediction, t=target.
    return ((y-t)**2).mean()



#--------------------------------------------------------------------------
def gradient(x, t, y):
    return np.mean(2*x*(y - t))



#--------------------------------------------------------------------------
for epoch in range(n_iters):
    # predict = forward pass
    Y = forward(X)

    # loss
    E = loss(T, Y)
    
    # calculate gradients
    dw = gradient(X, T, Y)

    # update weights
    w -= learning_rate * dw

    if epoch % 4 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {E:.8f}')



#--------------------------------------------------------------------------  
print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 5: w = 1.113, loss = 8.17471600
epoch 9: w = 1.537, loss = 2.22753215
epoch 13: w = 1.758, loss = 0.60698175
epoch 17: w = 1.874, loss = 0.16539653
epoch 21: w = 1.934, loss = 0.04506905
epoch 25: w = 1.966, loss = 0.01228092
epoch 29: w = 1.982, loss = 0.00334642
epoch 33: w = 1.991, loss = 0.00091188
epoch 37: w = 1.995, loss = 0.00024848
epoch 41: w = 1.997, loss = 0.00006770
epoch 45: w = 1.999, loss = 0.00001845
epoch 49: w = 1.999, loss = 0.00000503
Prediction after training: f(5) = 9.997
